## Data Loading

In [1]:
import torch
import torchvision

In [2]:
batch_size = 100

In [3]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5], [0.5])])

In [4]:
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)

In [5]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size ,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size ,shuffle=True)

In [6]:
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)

In [7]:
example_data.shape

torch.Size([100, 1, 28, 28])

In [8]:
"number of batches: {0}".format(len(list(examples)))

'number of batches: 99'

## Data View

In [9]:
import matplotlib.pyplot as plt

fig = plt.figure()
plt.imshow(example_data[0][0], cmap='gray', interpolation='none')
plt.title("Ground Truth: {}".format(example_targets[0]))

Text(0.5, 1.0, 'Ground Truth: 7')

## Generative Adversarial Network Definition

In [10]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [11]:
class Generator(nn.Module):
    
    def __init__(self, g_input_dim, g_output_dim, hidden_size=256):
        
        super(Generator, self).__init__()      
        
        self.layer = nn.Sequential(
                        nn.Linear(g_input_dim, hidden_size),
                        nn.LeakyReLU(0.2),
                        nn.Linear(hidden_size, hidden_size*2),
                        nn.LeakyReLU(0.2),
                        nn.Linear(hidden_size*2, hidden_size*4),
                        nn.LeakyReLU(0.2),
                        nn.Linear(hidden_size*4, g_output_dim))
        
        self.output = nn.Tanh()
    
    # forward method
    def forward(self, x): 
        x = self.layer(x)
        return self.output(x)

In [12]:
class Discriminator(nn.Module):
            
    def __init__(self, d_input_dim, hidden_size=1024):
            
        super(Discriminator, self).__init__()
            
        self.layer = nn.Sequential(
                        nn.Linear(d_input_dim, hidden_size),
                        nn.LeakyReLU(0.2),
                        nn.Dropout(0.3),       
                        nn.Linear(hidden_size, hidden_size//2),
                        nn.LeakyReLU(0.2),
                        nn.Dropout(0.3),       
                        nn.Linear(hidden_size//2, hidden_size//4),
                        nn.LeakyReLU(0.2),
                        nn.Dropout(0.3),
                        nn.Linear(hidden_size//4, 1))
            
        self.output = nn.Sigmoid()
    
    # forward method
    def forward(self, x):
        x = self.layer(x)
        return self.output(x)

## Pre-Training Parameters

In [13]:
z_dim = 100
mnist_dim = train_dataset.data.size(1) * train_dataset.data.size(2)

G = Generator(g_input_dim = z_dim, g_output_dim = mnist_dim)
D = Discriminator(mnist_dim)

In [14]:
G

Generator(
  (layer): Sequential(
    (0): Linear(in_features=100, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): LeakyReLU(negative_slope=0.2)
    (4): Linear(in_features=512, out_features=1024, bias=True)
    (5): LeakyReLU(negative_slope=0.2)
    (6): Linear(in_features=1024, out_features=784, bias=True)
  )
  (output): Tanh()
)

In [15]:
D

Discriminator(
  (layer): Sequential(
    (0): Linear(in_features=784, out_features=1024, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=1024, out_features=512, bias=True)
    (4): LeakyReLU(negative_slope=0.2)
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=512, out_features=256, bias=True)
    (7): LeakyReLU(negative_slope=0.2)
    (8): Dropout(p=0.3, inplace=False)
    (9): Linear(in_features=256, out_features=1, bias=True)
  )
  (output): Sigmoid()
)

In [16]:
# loss
loss = nn.BCELoss() 

# optimizer
lr = 0.0002 
G_optimizer = optim.Adam(G.parameters(), lr = lr)
D_optimizer = optim.Adam(D.parameters(), lr = lr)

## Training

In [17]:
n_epoch = 100

In [18]:
def G_train(x):
    
    G.zero_grad()

    z = Variable(torch.randn(batch_size, z_dim))
    y = Variable(torch.ones(batch_size, 1))

    G_output = G(z)
    D_output = D(G_output)
    G_loss = loss(D_output, y)

    G_loss.backward()
    G_optimizer.step()
        
    return G_loss.data.item()

In [19]:
def D_train(x):
    
    D.zero_grad()

    x_real, y_real = x.view(-1, mnist_dim), torch.ones(batch_size, 1)
    x_real, y_real = Variable(x_real), Variable(y_real)

    D_output = D(x_real)
    D_real_loss = loss(D_output, y_real)
    D_real_score = D_output

    z = Variable(torch.randn(batch_size, z_dim))
    x_fake, y_fake = G(z), Variable(torch.zeros(batch_size, 1))

    D_output = D(x_fake)
    D_fake_loss = loss(D_output, y_fake)
    D_fake_score = D_output

    D_loss = D_real_loss + D_fake_loss
    D_loss.backward()
    D_optimizer.step()
        
    return  D_loss.data.item()

In [20]:
for epoch in range(1, n_epoch+1):           
    D_losses, G_losses = [], []
    for batch_idx, (x, _) in enumerate(train_loader):
        D_losses.append(D_train(x))
        G_losses.append(G_train(x))

    print('[%d/%d]: loss_d: %.3f, loss_g: %.3f' % (
            (epoch), n_epoch, torch.mean(torch.FloatTensor(D_losses)), torch.mean(torch.FloatTensor(G_losses))))

[1/100]: loss_d: 0.761, loss_g: 4.021
[2/100]: loss_d: 0.984, loss_g: 1.987
[3/100]: loss_d: 0.932, loss_g: 2.087
[4/100]: loss_d: 0.779, loss_g: 2.110
[5/100]: loss_d: 0.502, loss_g: 2.907
[6/100]: loss_d: 0.555, loss_g: 2.623
[7/100]: loss_d: 0.597, loss_g: 2.502
[8/100]: loss_d: 0.560, loss_g: 2.512
[9/100]: loss_d: 0.649, loss_g: 2.350
[10/100]: loss_d: 0.686, loss_g: 2.257
[11/100]: loss_d: 0.696, loss_g: 2.273
[12/100]: loss_d: 0.731, loss_g: 2.091
[13/100]: loss_d: 0.792, loss_g: 1.965
[14/100]: loss_d: 0.778, loss_g: 2.004
[15/100]: loss_d: 0.802, loss_g: 1.889
[16/100]: loss_d: 0.791, loss_g: 1.968
[17/100]: loss_d: 0.797, loss_g: 1.900
[18/100]: loss_d: 0.830, loss_g: 1.879
[19/100]: loss_d: 0.834, loss_g: 1.804
[20/100]: loss_d: 0.872, loss_g: 1.732
[21/100]: loss_d: 0.869, loss_g: 1.702
[22/100]: loss_d: 0.926, loss_g: 1.608
[23/100]: loss_d: 0.905, loss_g: 1.654
[24/100]: loss_d: 0.941, loss_g: 1.590
[25/100]: loss_d: 0.938, loss_g: 1.566
[26/100]: loss_d: 0.953, loss_g: 1

## Save Model

In [22]:
torch.save(D.state_dict(), 'model/Discriminator.pth')
torch.save(G.state_dict(), 'model/Generator.pth')

In [ ]:
with torch.no_grad():
    test_z = Variable(torch.randn(batch_size, z_dim))
    generated = G(test_z)

    save_image(generated.view(generated.size(0), 1, 28, 28), './samples/sample_' + '.png')